# ADCS Simulator and System Modeling / Selection Notebook 
Follow the readme to install dependencies and correctly configure you environment/machine to run the varios blocks.

In [1]:
%matplotlib widget
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
import lib.physics as physics
import lib.cubesat as cubesat
import lib.renderer as renderer

In [ ]:
def vcircular(altitude):
    return math.sqrt(physics.Consts.mu / (altitude + physics.Consts.R_earth))

initial_state  = cubesat.State(physics.Consts.ISS_altitude + physics.Consts.R_earth, 0, 0, 0, vcircular(physics.Consts.ISS_altitude), 0) # x, y, z, dx, dy, dz
qubesat = cubesat.Cubesat(1, length=0.1, width=0.1, height=0.1) # mass in kg, length, width, height in m

planet  = physics.Planet(physics.Consts.M_earth, physics.Consts.R_earth) # mass in kg, radius in meters
# planet  = physics.Planet(10, 200) # mass in kg, radius in meters

In [10]:
class State:
    def __init__(self, x, y, z, dx, dy, dz):
        """
        Structure of the state vector (units are m and m/s)
        x, y, z, dx, dy, dz
        """
        self.state_vector = np.array([x, y, z, dx, dy, dz])
        
    def get_earth_referenced():
        # altitude, lat, long, v_altitude, v_lat, v_long, quaternion rotation
        return None
    
    def get_cubesat_referenced():
        # x, y, z, x_v, y_v, z_v,  quaternion rotation
        return None
    
    def get_cubesat_earth_transform():
        # homogeneous transformation matrix for the transformation between the qubesat and the earth
        return None
    
def state_from_vector(vector):
    return State(vector[0], vector[1], vector[2], vector[3], vector[4], vector[5])

In [30]:
class Simulator:
    def __init__(self, cubesat, planet, state, dt=0.1):
        """
        time and step size in seconds
        """
        self.dt = dt
        self.cubesat = cubesat
        self.planet = planet
        self.elapsed_time = 0
        self.state = state
        # functions that take in a state and returns a force vector <Fx,Fy,Fz> acting on the COM of the cubesat
        self.forces = []
        # functions that take in a state and returns an acceleration vector <ax,ay,az> acting on the COM of the cubesat
        self.accelerations = []
        
        # The new state is provided by the following equation
        # xk = Axk-1 + Buk-1 + wk-1
        # u is the vector sum of all the accelerations
        
        # 3D state matrix - A
        self.A = np.array([[1, 0, 0, self.dt, 0, 0],
                          [0, 1, 0, 0, self.dt, 0],
                          [0, 0, 1, 0, 0, self.dt],
                          [0, 0, 0, 1, 0, 0],
                          [0, 0, 0, 0, 1, 0],
                          [0, 0, 0, 0, 0, 1]]) 
        
        # 3D control matrix - B
        self.B = np.array([[1/2*self.dt**2, 0, 0],
                          [0, 1/2*self.dt**2, 0],
                          [0, 0, 1/2*self.dt**2],
                          [self.dt, 0, 0],
                          [0, self.dt, 0],
                          [0, 0, self.dt]])
        
    def step(self):
        # compute acceleration vector from gravity assuming points mass of the earth and satelite
        # [x,y,z]/|[x,y,z]| <- assumes point mass earth with center at (0,0,0)

        net_force = np.zeros(3)
        for forcer in self.forces:
            net_force += forcer(self.state, self.cubesat, self.planet)
            
        net_acc = np.zeros(3)
        for aaccelerator in self.accelerations:
            net_acc += aaccelerator(self.state, self.cubesat, self.planet)
            
        net_acc += net_force / self.cubesat.mass
        
        self.state = state_from_vector(np.dot(self.A, self.state.state_vector) + np.dot(self.B, net_acc))
        self.elapsed_time += self.dt
        return (self.elapsed_time, self.state)
    
    def add_forcer(self, forcer):
        self.forces.append(forcer)
        
    def add_accelerator(self, accelerator):
        self.accelerations.append(accelerator)

In [155]:
class XYZPlot:
    def __init__(self):
        pass
    
    def build(self, states, time_stamps, ax):
        x = [state.state_vector[0] for state in states]
        y = [state.state_vector[1] for state in states]
        z = [state.state_vector[2] for state in states]
        ax.plot(time_stamps, x, color="red", label='x')
        ax.plot(time_stamps, y, color="green", label='y')
        ax.plot(time_stamps, z, color="blue", label='z')
        ax.set_title("Position Plot")
        ax.set_xlabel("time (s)")
        ax.set_ylabel("position xyz (m)")
        
        # build a rectangle in axes coords
        left, width = .25, .5
        bottom, height = .25, .5
        right = left + width
        top = bottom + height
        
        ax.text(1, 0, "Hello there", fontsize=12, transform=ax.transAxes)
        handles, labels = ax.get_legend_handles_labels()
        ax.legend(handles, labels, loc='lower left')


class Renderer:
    def __init__(self):
        self.stop_condition = stop_condition
        self.time_stamps = []
        self.states = []
        self.plots = []
        
    def run(self, simulation, stop_condition):
        while not stop_condition(self.states, self.time_stamps):
            time, state = simulator.step()
            self.states.append(state)
            self.time_stamps.append(time)
            
    def add_plot(self, plot):
        self.plots.append(plot)
                    
    def render(self, figsize=(7,5), columns=3):
        fig = plt.figure(figsize=figsize)
        for i, plot in enumerate(self.plots):
            ax = fig.add_subplot((len(self.plots)+1)//columns if not columns == 1 else len(self.plots), columns, i+1)
            plot.build(self.states, self.time_stamps, ax)
        plt.show()

In [156]:
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def gravitational_force(state):
    r_hat = state.state_vector[:3]/np.linalg.norm(state.state_vector[:3])
    a = -physics.Consts.G * cubesat.mass * planet.mass / np.linalg.norm(state[:3])**2 * r_hat

def gravitational_forcer(state, cubesat, planet):
    return planet.get_gravitational_acceleration(state)


state = State(physics.Consts.ISS_altitude + physics.Consts.R_earth, 0, 0, 0, vcircular(physics.Consts.ISS_altitude), 0)
simulator = Simulator(qubesat, planet, state, 1)
simulator.add_accelerator(gravitational_forcer)

stop_condition = lambda states, times: len(states) > 10000
renderer = Renderer()
renderer.run(simulator, stop_condition)
plot1 = Plot()
renderer.add_plot(plot)

renderer.render(figsize=(5,5), columns=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# References
[Passive Magentic Attitude Control fro Cubesat Spacecraft](https://lasp.colorado.edu/home/csswe/files/2012/06/Gerhardt_SSC10_PMAC.pdf)

[Comparison of Control Laws for Magnetic Detumbling](https://www.researchgate.net/publication/263008407_Comparison_of_Control_Laws_for_Magnetic_Detumbling)
[]()


https://docs.google.com/document/d/16QsLCta1YLF__5pwcChfcjYQvqa9x6MZi19Ohj2Yuls/edit

https://www.youtube.com/playlist?list=PL_D7_GvGz-v3mDQ9iR-cfjXsQf4DeR1_H

https://pypi.org/project/pyIGRF/

https://stackoverflow.com/questions/53074908/map-an-image-onto-a-sphere-and-plot-3d-trajectories


In [54]:
x = [9, 10, 11, 12]
x_t = [0.1, 0.2, 0.3, 0.1]
y = [5, 6, 7, 8]
z = [1, 2, 3, 4]

In [84]:
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(231)
ax = fig.add_subplot(232)
ax = fig.add_subplot(233)
ax = fig.add_subplot(234)
ax = fig.add_subplot(235)






# ax.plot(x_t, x, color="red", label='x', linestyle='--')
# ax.plot(y, color="green", label='y')
# ax.plot(z, color="blue", label='z')
# ax.set_title("Here")
# ax.set_xlabel("time (s)")
# ax.set_ylabel("position (m)")
# handles, labels = ax.get_legend_handles_labels()
# ax.legend(handles, labels)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …